In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [70]:
# Install Google Play Scraper
!pip install google_play_scraper

In [71]:
# Install App Store Scraper
!pip install app-store-scraper

In [72]:
# Imports
import json
from tqdm import tqdm
from datetime import datetime, date, timedelta
from pprint import pprint

# Google Play
from google_play_scraper import Sort, reviews, app

# App Store
from app_store_scraper import AppStore

## Scraping Reviews from Google Play Store

In [85]:
# Specify the app
app_packages=['com.square_enix.android_googleplay.FFVIII']

# Initialize a blank list to hold all the reviews
playstore_reviews=[]

In [86]:
# Looping to scrape the reviews
for ap in tqdm(app_packages):
    # Low score = 1, high score = 5
    for score in list(range(1, 6)):
        # Sort from new to old
        for sort_order in [Sort.NEWEST]:
            # Review components
            rvs, _ = reviews(ap, 
                                 sort = sort_order, 
                                 count = 1000, 
                                 filter_score_with = score)
            # Iterate through reviews gathered
            for r in rvs:
                r['sortOrder'] = 'newest'
                r['AppId'] = ap
            # Add new review to list
            playstore_reviews.extend(rvs)

100%|██████████| 1/1 [00:00<00:00,  1.14it/s]


In [87]:
# Check gathered contents
playstore_reviews[50]

{'reviewId': '6f2edd4f-ad80-483e-a41b-78b060882dfa',
 'userName': 'Brian Howard',
 'userImage': 'https://play-lh.googleusercontent.com/a-/AFdZucop-pg-cjtJw5eEtc5bAP4wnYeSzz9pKuDH61c',
 'content': 'App doesnt work right',
 'score': 1,
 'thumbsUpCount': 0,
 'reviewCreatedVersion': '1.0.0 (210309)',
 'at': datetime.datetime(2021, 4, 19, 18, 4, 49),
 'replyContent': None,
 'repliedAt': None,
 'sortOrder': 'newest',
 'AppId': 'com.square_enix.android_googleplay.FFVIII'}

From the above display it appears that all of the relevant data has been collected. 

In [88]:
# Create a dataframe only for reviews from the Google Play Store
google_play_reviews = pd.DataFrame(playstore_reviews)

# Drop duplicates
google_play_reviews = google_play_reviews.drop_duplicates(keep = 'first')

# Check
google_play_reviews.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,AppId
0,5ba25ca1-6bb0-4a28-8db5-b9286f46d5cc,Jack Agresti,https://play-lh.googleusercontent.com/a-/AFdZu...,"Played FFVII and IX, they played just fine. My...",1,0,1.0.1,2022-07-26 11:44:45,None,None,newest,com.square_enix.android_googleplay.FFVIII
1,4803f991-793b-4574-af77-52b3c7c51099,Justice Peralta,https://play-lh.googleusercontent.com/a-/AFdZu...,Bad aspect ratio and flubbed on-screen controls.,1,0,None,2022-07-16 09:06:41,None,None,newest,com.square_enix.android_googleplay.FFVIII
2,8b215996-c5d8-4cd5-8b43-e2dbf7da2c24,Brian Puckett,https://play-lh.googleusercontent.com/a-/AFdZu...,"Game runs great for about 5 minutes, then you ...",1,0,1.0.1,2022-07-09 21:07:57,None,None,newest,com.square_enix.android_googleplay.FFVIII
3,e1d302de-30fa-47f3-bb49-136cc378bc84,B B,https://play-lh.googleusercontent.com/a/AItbvm...,Any updates on this? Why can I install final f...,1,1,1.0.1,2022-06-17 18:15:38,None,None,newest,com.square_enix.android_googleplay.FFVIII
4,c05044c7-35b3-4f3b-8d5d-6fe733f006ee,James Small,https://play-lh.googleusercontent.com/a-/AFdZu...,My favorite FF but this seriously needs fixing...,1,2,1.0.1,2022-05-29 17:13:56,None,None,newest,com.square_enix.android_googleplay.FFVIII


In [89]:
# Get information about the gathered reviews
google_play_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335 entries, 0 to 334
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              335 non-null    object        
 1   userName              335 non-null    object        
 2   userImage             335 non-null    object        
 3   content               335 non-null    object        
 4   score                 335 non-null    int64         
 5   thumbsUpCount         335 non-null    int64         
 6   reviewCreatedVersion  299 non-null    object        
 7   at                    335 non-null    datetime64[ns]
 8   replyContent          0 non-null      object        
 9   repliedAt             0 non-null      object        
 10  sortOrder             335 non-null    object        
 11  AppId                 335 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(9)
memory usage: 34.0+ KB


In [90]:
# Summarize the unique values for specific columns
gp_cols = google_play_reviews.columns

# Iterate through the list of columns
for c in range(1,len(gp_cols)):
    if (gp_cols[c] == 'score') | (gp_cols[c] == 'reviewCreatedVersion') | (gp_cols[c] == 'replyContent') | (gp_cols[c] == 'repliedAt'):
        # Print the unique values per column
        print(f"{gp_cols[c]} unique values: {google_play_reviews[gp_cols[c]].unique()}")

score unique values: [1 2 3 4 5]
reviewCreatedVersion unique values: ['1.0.1' None '1.0.0 (210309)']
replyContent unique values: [None]
repliedAt unique values: [None]


## Scraping Reviews from AppStore

In [79]:
# Specify the app reviews to scrape
ff8_app = AppStore(country="us", app_name="final-fantasy-viii-remastered", app_id=1326740784)

# Specify the number of reviews
ff8_app.review(how_many=225, sleep=25)

# Show how many reviews were actually received
print(f"There were {ff8_app.reviews_count} reviews scraped.")

There were 98 reviews scraped.


In [80]:
# Data frame of only App Store reviews
app_store_reviews=pd.DataFrame(ff8_app.reviews)

# Drop duplicates
app_store_reviews = app_store_reviews.drop_duplicates(keep='first')

# Check
app_store_reviews.head()

,rating,isEdited,review,userName,title,date
0,4,False,I have been waiting for this to come out for i...,Rxdrummer,"Great Port, with One Flaw",2021-05-01 19:31:13
1,5,False,"Wow, what a trip down memory lane! I’ve been w...",KittenMoney,"A million times, thank you!",2021-03-25 12:47:16
2,2,False,This is by far my favorite final fantasy out o...,Sirdracoblight,"Favorite Final Fantasy, but underwhelmed.",2021-09-18 04:11:58
3,3,False,It's difficult to give this a fair review part...,IsthereAnicknameathatIsntTaken,Having trouble starting,2021-03-29 16:12:20
4,3,False,This game is an excellent port of a classic RP...,ianfloresta,"Excellent, but desperately needs controller su...",2021-04-26 22:26:38


In [81]:
# Add the app id as an identifier once the dataframes are merged
app_store_reviews['AppId'] = 1326740784

# Check
app_store_reviews.head()

,rating,isEdited,review,userName,title,date,AppId
0,4,False,I have been waiting for this to come out for i...,Rxdrummer,"Great Port, with One Flaw",2021-05-01 19:31:13,1326740784
1,5,False,"Wow, what a trip down memory lane! I’ve been w...",KittenMoney,"A million times, thank you!",2021-03-25 12:47:16,1326740784
2,2,False,This is by far my favorite final fantasy out o...,Sirdracoblight,"Favorite Final Fantasy, but underwhelmed.",2021-09-18 04:11:58,1326740784
3,3,False,It's difficult to give this a fair review part...,IsthereAnicknameathatIsntTaken,Having trouble starting,2021-03-29 16:12:20,1326740784
4,3,False,This game is an excellent port of a classic RP...,ianfloresta,"Excellent, but desperately needs controller su...",2021-04-26 22:26:38,1326740784


In [82]:
# Get basic info about the dataframe
app_store_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98 entries, 0 to 97
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   rating    98 non-null     int64         
 1   isEdited  98 non-null     bool          
 2   review    98 non-null     object        
 3   userName  98 non-null     object        
 4   title     98 non-null     object        
 5   date      98 non-null     datetime64[ns]
 6   AppId     98 non-null     int64         
dtypes: bool(1), datetime64[ns](1), int64(2), object(3)
memory usage: 5.5+ KB


In [83]:
# Unique values
print(app_store_reviews.rating.unique())
print(app_store_reviews.isEdited.unique())

[4 5 2 3 1]
[False]


There is some similarity between the data returned by the two different scrapers.

Google Play Results   |   App Store Results
----------------------|--------------------
userName              |   userName
content               |   review
score                 |   rating
at                    |   date
AppId                 |   AppId

The extra columns from the dataframe of google play results will be dropped, in addition to any unneeded columns from both dataframes:
- userName
- isEdited
- replyContent
- repliedAt
- sortOrder
- thumbsUpCount
- reviewCreatedVersion
- title

The remaining columns between the two data frames will then be renamed.
- content --> review
- score --> rating
- at --> date

## Cleaning up the Data Frames

In [91]:
# Google play dataframe
google_play_reviews.drop(['userName', 'replyContent', 'repliedAt', 'reviewId', 'userImage', 'thumbsUpCount','reviewCreatedVersion', 'sortOrder'], axis=1, inplace=True)

# Check
google_play_reviews.head()

,content,score,at,AppId
0,"Played FFVII and IX, they played just fine. My...",1,2022-07-26 11:44:45,com.square_enix.android_googleplay.FFVIII
1,Bad aspect ratio and flubbed on-screen controls.,1,2022-07-16 09:06:41,com.square_enix.android_googleplay.FFVIII
2,"Game runs great for about 5 minutes, then you ...",1,2022-07-09 21:07:57,com.square_enix.android_googleplay.FFVIII
3,Any updates on this? Why can I install final f...,1,2022-06-17 18:15:38,com.square_enix.android_googleplay.FFVIII
4,My favorite FF but this seriously needs fixing...,1,2022-05-29 17:13:56,com.square_enix.android_googleplay.FFVIII


In [92]:
# App store dataframe
app_store_reviews.drop(['isEdited', 'userName', 'title'], axis=1, inplace=True)

# Check
app_store_reviews.head()

,rating,review,date,AppId
0,4,I have been waiting for this to come out for i...,2021-05-01 19:31:13,1326740784
1,5,"Wow, what a trip down memory lane! I’ve been w...",2021-03-25 12:47:16,1326740784
2,2,This is by far my favorite final fantasy out o...,2021-09-18 04:11:58,1326740784
3,3,It's difficult to give this a fair review part...,2021-03-29 16:12:20,1326740784
4,3,This game is an excellent port of a classic RP...,2021-04-26 22:26:38,1326740784
